In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
T = 4
Delta = 800
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]
print(cp_truth)

n = 20

[ 800 1600 2400]


In [7]:
path = '/content/drive/MyDrive/CPD_BT/experiment_random/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    beta_list = np.load(f)
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [8]:
X_train_list.shape

(100, 3200, 20)

In [9]:
np.random.seed(0)

grid_n = 90
gamma_list = [20, 40]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_d = np.zeros(B)
loc_error_d = np.zeros(B)
K_d = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    dp_fit = dplr_cv_bt(grid_n, lam_list, gamma_list)
    cp_best, param_best, cp_best_cand = dp_fit.fit((Y_train, X_train), (Y_test, X_test))    
    run_time_d[b] = time.time() - start_time
    loc_error_d[b] = cp_distance(cp_best, cp_truth)
    K_d[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_d.mean(), run_time_d.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_d.std(), run_time_d.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_d < T - 1), sum(K_d == T - 1), sum(K_d > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 23.89, avg time: 105.77146171569824
std loc error: 27.626398607129378, std time: 4.222208002819576
K < K*: 0, K = K*: 100, K > K*: 0


In [10]:
loc_error_d

array([ 17.,   7.,   5.,  13.,  21.,   1.,   8.,  28.,   5.,   7.,  22.,
       102.,  23.,   9.,  16.,   7.,   5.,  17.,  16.,  10.,   9.,  12.,
         6.,  61.,  15.,   4.,  24.,  12.,  16.,  64.,   3.,  19., 197.,
        10.,   7.,  28.,   7.,  44.,  24.,  13.,  29.,   5.,   4.,   7.,
        20.,   3.,  29.,  13.,  24.,  37.,   7., 103.,  13., 115.,  18.,
        21.,  21.,   9.,  59.,   1.,  48.,  29.,  30.,  11.,   9.,  47.,
        28.,   4.,   7.,   3.,  35.,  39.,  20.,  20.,  35.,  30.,   1.,
         9.,  15.,  29.,  29.,  29.,  24.,  31.,  20.,   8.,   6.,  26.,
        44.,  21.,   7.,  21.,  20.,   7.,  15.,   2.,  64.,  84.,  22.,
         8.])

In [11]:
cp_best_list

[[817, 1593, 2401],
 [807, 1606, 2403],
 [795, 1603, 2398],
 [799, 1597, 2413],
 [798, 1598, 2379],
 [801, 1600, 2399],
 [808, 1602, 2404],
 [772, 1602, 2382],
 [805, 1598, 2400],
 [807, 1604, 2399],
 [778, 1602, 2400],
 [795, 1598, 2502],
 [823, 1588, 2391],
 [793, 1591, 2401],
 [784, 1601, 2403],
 [793, 1597, 2395],
 [800, 1595, 2400],
 [795, 1600, 2417],
 [811, 1585, 2416],
 [810, 1601, 2405],
 [807, 1591, 2400],
 [798, 1601, 2388],
 [794, 1594, 2402],
 [787, 1594, 2339],
 [785, 1601, 2397],
 [804, 1599, 2400],
 [799, 1576, 2408],
 [812, 1599, 2395],
 [816, 1616, 2400],
 [800, 1586, 2464],
 [802, 1601, 2403],
 [809, 1604, 2419],
 [997, 1588, 2408],
 [810, 1594, 2408],
 [793, 1603, 2395],
 [796, 1603, 2428],
 [793, 1605, 2403],
 [792, 1602, 2444],
 [824, 1586, 2386],
 [798, 1592, 2413],
 [800, 1629, 2403],
 [802, 1605, 2396],
 [804, 1601, 2400],
 [798, 1607, 2394],
 [800, 1601, 2380],
 [799, 1603, 2400],
 [829, 1586, 2403],
 [800, 1590, 2387],
 [824, 1598, 2390],
 [763, 1603, 2406],


In [12]:
cp_best_cand_list

[array([ 805, 1610, 2380]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 2380]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 2380]),
 array([ 770, 1610, 2380]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 2415]),
 array([ 770, 1610, 2415]),
 array([ 805, 1610, 2485]),
 array([ 805, 1610, 2380]),
 array([ 805, 1575, 2415]),
 array([ 805, 1610, 2380]),
 array([ 805, 1575, 2380]),
 array([ 805, 1575, 2415]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 2415]),
 array([ 805, 1575, 2415]),
 array([ 805, 1610, 2380]),
 array([ 805, 1610, 2380]),
 array([ 805, 1610, 2345]),
 array([ 770, 1610, 2415]),
 array([ 805, 1610, 2380]),
 array([ 805, 1610, 2415]),
 array([ 770, 1610, 2415]),
 array([ 805, 1610, 2415]),
 array([ 805, 1575, 2450]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 2415]),
 array([ 840, 1575, 2415]),
 array([ 805, 1575, 2415]),
 array([ 805, 1610, 2415]),
 array([ 805, 1610, 

In [13]:
import pickle
with open(path + 'dplr_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_d, run_time_d, K_d], f)